In [1]:
import random
from pathlib import Path
import tarfile
from typing import Any
from logging import INFO
from collections import defaultdict, OrderedDict
from collections.abc import Sequence, Callable
import numbers
import copy

import numpy as np
import torch
from torch import nn
from torch.nn import Module
from torch.utils.data import DataLoader, Dataset
from enum import IntEnum
import flwr
from flwr.server import History, ServerConfig
from flwr.server.strategy import FedAvgM as FedAvg, Strategy
from c2m3.flower.fed_frank_wolfe_strategy import FrankWolfeSync
from flwr.common import log, NDArrays, Scalar, Parameters, ndarrays_to_parameters
from flwr.client.client import Client
from c2m3.match.utils import apply_permutation_to_statedict
from c2m3.match.frank_wolfe_sync_matching import frank_wolfe_synchronized_matching
from c2m3.match.permutation_spec import CNNPermutationSpecBuilder
from c2m3.modules.pl_module import MyLightningModule
NUM_CLASSES_FEMNIST = 62

from c2m3.common.client_utils import (
    Net,
    load_femnist_dataset,
    get_network_generator_cnn as get_network_generator,
    train_femnist,
    test_femnist,
    save_history,
    get_model_parameters,
    set_model_parameters
)


# Add new seeds here for easy autocomplete
class Seeds(IntEnum):
    """Seeds for reproducibility."""

    DEFAULT = 1337


np.random.seed(Seeds.DEFAULT)
random.seed(Seeds.DEFAULT)
torch.manual_seed(Seeds.DEFAULT)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


PathType = Path | str | None


def get_device() -> str:
    """Get the device (cuda, mps, cpu)."""
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
        device = "mps"
    return device

2025-03-15 16:59:26.782231: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-15 16:59:26.848236: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-15 16:59:27.666153: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/dawn/venvs/fed/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-15 16:59:29,300	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
home_dir = Path.cwd() / ".."
dataset_dir: Path = home_dir / "femnist"
data_dir: Path = dataset_dir / "data"
centralized_partition: Path = dataset_dir / "client_data_mappings" / "centralized"
centralized_mapping: Path = dataset_dir / "client_data_mappings" / "centralized" / "0"
federated_partition: Path = dataset_dir / "client_data_mappings" / "fed_natural"

In [3]:
# Set random seed for reproducibility
torch.manual_seed(33)

n = Net()

# Create a copy of the network
n_permuted = copy.deepcopy(n)

# Get the correct layer sizes from the network
first_layer_size = n.conv1.out_channels  # Should be 6
second_layer_size = n.conv2.out_channels  # Should be 16
third_layer_size = n.fc1.out_features  # Should be 120
fourth_layer_size = n.fc2.out_features  # Should be 84
fifth_layer_size = n.fc3.out_features  # Should be 10
print(f'{first_layer_size=}\n{second_layer_size=}\n{third_layer_size=}\n{fourth_layer_size=}\n{fifth_layer_size=}')

first_layer_size=6
second_layer_size=16
third_layer_size=120
fourth_layer_size=84
fifth_layer_size=62


In [4]:
# Random permutations
perm_matrices_old = {
    # 'P_conv1': torch.randperm(6),  # For first conv layer
    'P_conv1': torch.arange(6),  # For first conv layer
    'P_fc1': torch.randperm(120),  # For first FC layer
    #'P_fc1': torch.arange(120),  # For first FC layer
    'P_fc2': torch.randperm(84)    # For second FC layer
    #'P_fc2': torch.arange(84)    # For second FC layer
}

# Set up parameters for frank_wolfe_synchronized_matching
mlm = MyLightningModule(n, num_classes=NUM_CLASSES_FEMNIST)
ref_model = copy.deepcopy(mlm)

permutation_spec_builder = CNNPermutationSpecBuilder()
perm_spec = permutation_spec_builder.create_permutation_spec(ref_model=ref_model)

# Use the permutation spec to apply them correctly
permuted_state_dict = apply_permutation_to_statedict(perm_spec, perm_matrices_old, n_permuted.state_dict())
n_permuted.load_state_dict(permuted_state_dict)

mlm_permuted = MyLightningModule(n_permuted, num_classes=NUM_CLASSES_FEMNIST)
models = {
    'a': mlm,
    'b': mlm_permuted
}

/home/dawn/venvs/fed/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:268: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(


In [7]:
# This illustrates one of the dependency cycles in permutation matrix a.
# 17 -> 30 -> 79 -> 106 -> 17
# This dependency is NOT fixed by the C2M3 algorithm, see below.
# However the other dependency cycles are fixed.
print(f'{perm_matrices_old["P_fc1"][17]=}')
print(f'{perm_matrices_old["P_fc1"][30]=}')
print(f'{perm_matrices_old["P_fc1"][79]=}')
print(f'{perm_matrices_old["P_fc1"][106]=}')

perm_matrices_old["P_fc1"][17]=tensor(30)
perm_matrices_old["P_fc1"][30]=tensor(79)
perm_matrices_old["P_fc1"][79]=tensor(106)
perm_matrices_old["P_fc1"][106]=tensor(17)


In [8]:
# Get the inverse permutation matrices by reversing the indices
inverse_perm_matrices = {}
for key in ['P_conv1', 'P_fc1', 'P_fc2']:
    inverse_perm_matrices[key] = torch.argsort(perm_matrices_old[key])

# Apply inverse permutations to n_permuted
unpermuted_state_dict = apply_permutation_to_statedict(perm_spec, inverse_perm_matrices, n_permuted.state_dict())

# Create a new network to hold the unpermuted weights
n_unpermuted = copy.deepcopy(n_permuted)
n_unpermuted.load_state_dict(unpermuted_state_dict)

# Check if unpermuted network matches original
print("\nChecking if unpermuted network matches original:")
print("Conv1 weight matches? ", torch.allclose(n.conv1.weight.data, n_unpermuted.conv1.weight.data))
print("Conv1 bias matches? ", torch.allclose(n.conv1.bias.data, n_unpermuted.conv1.bias.data))
print("Conv2 weight matches? ", torch.allclose(n.conv2.weight.data, n_unpermuted.conv2.weight.data))
print("Conv2 bias matches? ", torch.allclose(n.conv2.bias.data, n_unpermuted.conv2.bias.data))
print("FC1 weight matches? ", torch.allclose(n.fc1.weight.data, n_unpermuted.fc1.weight.data))
print("FC1 bias matches? ", torch.allclose(n.fc1.bias.data, n_unpermuted.fc1.bias.data))
print("FC2 weight matches? ", torch.allclose(n.fc2.weight.data, n_unpermuted.fc2.weight.data))
print("FC2 bias matches? ", torch.allclose(n.fc2.bias.data, n_unpermuted.fc2.bias.data))
print("FC3 weight matches? ", torch.allclose(n.fc3.weight.data, n_unpermuted.fc3.weight.data))
print("FC3 bias matches? ", torch.allclose(n.fc3.bias.data, n_unpermuted.fc3.bias.data))



Checking if unpermuted network matches original:
Conv1 weight matches?  True
Conv1 bias matches?  True
Conv2 weight matches?  True
Conv2 bias matches?  True
FC1 weight matches?  True
FC1 bias matches?  True
FC2 weight matches?  True
FC2 bias matches?  True
FC3 weight matches?  True
FC3 bias matches?  True


In [9]:
# Check if layers have been permuted by comparing sorted vs unsorted values
print("\nChecking if layers were permuted:")
print("Conv1 weight permuted? ", not torch.allclose(n.conv1.weight.data, n_permuted.conv1.weight.data))
print("Conv1 bias permuted? ", not torch.allclose(n.conv1.bias.data, n_permuted.conv1.bias.data))
print("Conv2 weight permuted? ", not torch.allclose(n.conv2.weight.data, n_permuted.conv2.weight.data))
print("Conv2 bias permuted? ", not torch.allclose(n.conv2.bias.data, n_permuted.conv2.bias.data))
print("FC1 weight permuted? ", not torch.allclose(n.fc1.weight.data, n_permuted.fc1.weight.data))
print("FC1 bias permuted? ", not torch.allclose(n.fc1.bias.data, n_permuted.fc1.bias.data))
print("FC2 weight permuted? ", not torch.allclose(n.fc2.weight.data, n_permuted.fc2.weight.data))
print("FC2 bias permuted? ", not torch.allclose(n.fc2.bias.data, n_permuted.fc2.bias.data))
print("FC3 weight permuted? ", not torch.allclose(n.fc3.weight.data, n_permuted.fc3.weight.data))
print("FC3 bias permuted? ", not torch.allclose(n.fc3.bias.data, n_permuted.fc3.bias.data))



Checking if layers were permuted:
Conv1 weight permuted?  False
Conv1 bias permuted?  False
Conv2 weight permuted?  False
Conv2 bias permuted?  False
FC1 weight permuted?  True
FC1 bias permuted?  True
FC2 weight permuted?  True
FC2 bias permuted?  True
FC3 weight permuted?  True
FC3 bias permuted?  False


In [10]:
print("Parameter shapes:")
for key, value in n.state_dict().items():
    print(f"{key}: {value.shape}")

Parameter shapes:
conv1.weight: torch.Size([6, 1, 5, 5])
conv1.bias: torch.Size([6])
conv2.weight: torch.Size([16, 6, 5, 5])
conv2.bias: torch.Size([16])
fc1.weight: torch.Size([120, 256])
fc1.bias: torch.Size([120])
fc2.weight: torch.Size([84, 120])
fc2.bias: torch.Size([84])
fc3.weight: torch.Size([62, 84])
fc3.bias: torch.Size([62])


In [11]:
params = {symbol: module.model.state_dict() for symbol, module in models.items()}
models_permuted_to_universe = {symbol: copy.deepcopy(model) for symbol, model in models.items()}
symbols = ['a', 'b']
merged_model = copy.deepcopy(models[symbols[0]])

perm_matrices = frank_wolfe_synchronized_matching(
    params=params,
    perm_spec=perm_spec,  # Need to define permutation spec
    symbols=['a', 'b'],
    combinations=[('a', 'b')],
    max_iter=500,
    initialization_method='identity',
    keep_soft_perms=False,
    device='cpu',
    verbose=True
)

Weight matching:   0%|          | 0/500 [00:00<?, ?it/s]

2025-03-15 17:00:30 INFO     Iteration 0                                    ]8;id=614242;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py\c2m3.match.frank_wolfe_sync_matching]8;;\:]8;id=767447;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py#66\66]8;;\

                    INFO     Step size: 0.500000006197539, function value: ]8;id=665713;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py\c2m3.match.frank_wolfe_sync_matching]8;;\:]8;id=827969;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py#111\111]8;;\
                             -45.228957045823336, num called functions: 6                                          

                    INFO     Objective: 45.228957                          ]8;id=214901;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py\c2m3.match.frank_wolfe_sync_matching]8;;\:]8;id=798528;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py#119\119]8;;\

                    INFO     Iteration 1                                    ]8;id=114597;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py\c2m3.match.frank_wolfe_sync_matching]8;;\:]8;id=446156;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py#66\66]8;;\

...

                    INFO     Objective: 94.322876                          ]8;id=130395;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py\c2m3.match.frank_wolfe_sync_matching]8;;\:]8;id=669977;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py#119\119]8;;\

                    INFO     Iteration 481                                  ]8;id=641504;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py\c2m3.match.frank_wolfe_sync_matching]8;;\:]8;id=508795;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py#66\66]8;;\

                    INFO     Step size: 0.000384372363517672, function     ]8;id=43443;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py\c2m3.match.frank_wolfe_sync_matching]8;;\:]8;id=453239;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py#111\111]8;;\
                             value: -94.3228758983314, num called                                                  
                             functions: 15                                                                         

                    INFO     Objective: 94.322876                          ]8;id=691254;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py\c2m3.match.frank_wolfe_sync_matching]8;;\:]8;id=85455;file:///home/dawn/repos/c2m3-federated/c2m3/match/frank_wolfe_sync_matching.py#119\119]8;;\

Weight matching:  96%|█████████▌| 481/500 [00:40<00:01, 11.75it/s]


In [12]:
print(f'{perm_matrices[0]["a"]["P_conv1"]=}')
print(f'{perm_matrices[0]["a"]["P_fc1"]=}')
print(f'{perm_matrices[0]["a"]["P_fc2"]=}')

print(f'{perm_matrices[0]["b"]["P_conv1"]=}')
print(f'{perm_matrices[0]["b"]["P_fc1"]=}')
print(f'{perm_matrices[0]["b"]["P_fc2"]=}')

perm_matrices[0]["a"]["P_conv1"]=tensor([0, 1, 2, 3, 4, 5])
perm_matrices[0]["a"]["P_fc1"]=tensor([  0,  29,   3,  67, 107,  49,   6,   7,  68,   9,  16,  76, 103, 119,
         54,  15,  28,  17,  65,  63,  59,  98,  92,  80,  85,  37,  70,  55,
         66,  50,  30,  31, 114,  11,  41,  21,  99,  74,  38,  22,  93,  90,
         81,  43,  69,   1,  45,  48,  36,  40,  10, 101,  13,  94,  95,  52,
         19, 109,  83,  42,  44, 116, 100, 113,  64,  72,  61,  89,  34,  84,
        104,  47,  77, 112,  39,  73,  78, 108,  60,  79, 102,  96, 115,  62,
         91,   2,  53,  97,  33, 110,  46,   8, 105,  86, 118,  88,  75,  12,
         58,  57,   4,  82,  14,  51, 117,  35, 106,  32,  25,  56, 111,  18,
         24,  23,  20,   5,  87,  71,  27,  26])
perm_matrices[0]["a"]["P_fc2"]=tensor([ 0,  1, 41, 45,  4, 43,  6,  7, 16, 79, 10, 48, 11, 67, 14, 25, 73, 68,
        18, 39, 20, 21, 22, 23, 63,  5, 26, 27, 28, 29, 15, 31, 32, 33, 34, 35,
        36, 12, 38, 71, 78, 17, 42, 51, 44,  

In [13]:
# Apply permutation matrices to the parameters of n and n_permuted
from c2m3.match.utils import perm_indices_to_perm_matrix, perm_matrix_to_perm_indices

n2 = copy.deepcopy(n)
n_permuted2 = copy.deepcopy(n_permuted)

# Process both models consistently
for model_symbol, model in [('a', n2), ('b', n_permuted2)]:
    perms_to_apply = {}
    
    for perm_name in perm_matrices[0][model_symbol].keys():
        # Get original permutation indices
        perm = perm_matrices[0][model_symbol][perm_name]
        
        # Convert to matrix, transpose, then back to indices
        perm_matrix = perm_indices_to_perm_matrix(perm).T
        perm_to_apply = perm_matrix_to_perm_indices(perm_matrix)
        
        perms_to_apply[perm_name] = perm_to_apply
    
    # Apply the permutation to the model
    updated_params = apply_permutation_to_statedict(
        perm_spec, perms_to_apply, model.state_dict()
    )
    model.load_state_dict(updated_params)

# Check if both results are the same
for layer in n2.state_dict():
    if not torch.allclose(n2.state_dict()[layer], n_permuted2.state_dict()[layer], rtol=1e-4, atol=1e-4):
        print(f'{layer} not the same')
        
        # Get the tensors
        tensor1 = n2.state_dict()[layer]
        tensor2 = n_permuted2.state_dict()[layer]
        
        # Check if this is a weight matrix with rows (2D or 4D tensor)
        if len(tensor1.shape) in [2, 4]:
            # For 4D convolutional weights, reshape to 2D for comparison
            if len(tensor1.shape) == 4:
                t1_reshaped = tensor1.reshape(tensor1.shape[0], -1)
                t2_reshaped = tensor2.reshape(tensor2.shape[0], -1)
            else:
                t1_reshaped = tensor1
                t2_reshaped = tensor2
                
            # Calculate row-wise cosine similarity
            for i in range(t1_reshaped.shape[0]):
                row_sim = torch.cosine_similarity(
                    t1_reshaped[i].flatten().unsqueeze(0), 
                    t2_reshaped[i].flatten().unsqueeze(0), 
                    dim=1
                )
                
                # Find if this row is similar to any row in the other tensor
                all_sims = []
                for j in range(t2_reshaped.shape[0]):
                    sim = torch.cosine_similarity(
                        t1_reshaped[i].flatten().unsqueeze(0),
                        t2_reshaped[j].flatten().unsqueeze(0),
                        dim=1
                    )
                    all_sims.append((j, sim.item()))
                
                # Sort by similarity and check if the best match is the same index
                all_sims.sort(key=lambda x: x[1], reverse=True)
                best_match_idx, best_match_sim = all_sims[0]
                
                # Only print details if the best match is not the same index
                # or if the similarity is below threshold
                if best_match_idx != i or row_sim < 0.99:
                    if row_sim < 0.99:  # Threshold for obviously different rows
                        print(f'  Row {i} has low similarity: {row_sim.item():.5f}')
                    
                    # Only print best matches if the best match isn't the same index
                    if best_match_idx != i:
                        print(f"  Row {i} best matches in other tensor:")
                        for j, sim in all_sims[:3]:
                            print(f"    Row {j}: similarity {sim:.5f}")
            
            # Now check column-wise similarity
            if len(tensor1.shape) == 2:  # Only for 2D matrices
                # Calculate column-wise cosine similarity
                for i in range(t1_reshaped.shape[1]):
                    col_sim = torch.cosine_similarity(
                        t1_reshaped[:, i].unsqueeze(0), 
                        t2_reshaped[:, i].unsqueeze(0), 
                        dim=1
                    )
                    
                    # Find if this column is similar to any column in the other tensor
                    all_sims = []
                    for j in range(t2_reshaped.shape[1]):
                        sim = torch.cosine_similarity(
                            t1_reshaped[:, i].unsqueeze(0),
                            t2_reshaped[:, j].unsqueeze(0),
                            dim=1
                        )
                        all_sims.append((j, sim.item()))
                    
                    # Sort by similarity and check if the best match is the same index
                    all_sims.sort(key=lambda x: x[1], reverse=True)
                    best_match_idx, best_match_sim = all_sims[0]
                    
                    # Only print details if the best match is not the same index
                    # or if the similarity is below threshold
                    if best_match_idx != i or col_sim < 0.99:
                        if col_sim < 0.99:  # Threshold for obviously different columns
                            print(f'  Column {i} has low similarity: {col_sim.item():.5f}')
                        
                        # Only print best matches if the best match isn't the same index
                        if best_match_idx != i:
                            print(f"  Column {i} best matches in other tensor:")
                            for j, sim in all_sims[:3]:
                                print(f"    Column {j}: similarity {sim:.5f}")
        
        print(f'n2[{layer}]={n2.state_dict()[layer]}')
        print(f'n_permuted2[{layer}]={n_permuted2.state_dict()[layer]}')
    else:
        print(f'{layer} is the same')
    similarity = torch.cosine_similarity(n2.state_dict()[layer].flatten(), n_permuted2.state_dict()[layer].flatten(), dim=0)
    print(f'{layer} similarity: {similarity:.5f})')



conv1.weight is the same
conv1.weight similarity: 1.00000)
conv1.bias is the same
conv1.bias similarity: 1.00000)
conv2.weight is the same
conv2.weight similarity: 1.00000)
conv2.bias is the same
conv2.bias similarity: 1.00000)
fc1.weight not the same
  Row 17 has low similarity: 0.08021
  Row 17 best matches in other tensor:
    Row 106: similarity 1.00000
    Row 40: similarity 0.17030
    Row 51: similarity 0.14671
  Row 30 has low similarity: 0.10414
  Row 30 best matches in other tensor:
    Row 17: similarity 1.00000
    Row 67: similarity 0.13691
    Row 77: similarity 0.12938
  Row 79 has low similarity: -0.08946
  Row 79 best matches in other tensor:
    Row 30: similarity 1.00000
    Row 4: similarity 0.12599
    Row 50: similarity 0.12106
  Row 106 has low similarity: -0.11349
  Row 106 best matches in other tensor:
    Row 79: similarity 1.00000
    Row 119: similarity 0.15562
    Row 43: similarity 0.12833
  Column 0 has low similarity: 0.94036
  Column 1 has low similarit

In [14]:
# Apply permutation matrices to the parameters of n and n_permuted
from c2m3.match.utils import perm_indices_to_perm_matrix, perm_matrix_to_perm_indices

n3 = copy.deepcopy(n)
n_permuted3 = copy.deepcopy(n_permuted)

# First transform n_permuted3 to universe space using transpose of its perm matrix
perms_to_apply = {}
for perm_name in perm_matrices[0]['b'].keys():
    # Get original permutation indices
    perm = perm_matrices[0]['b'][perm_name]
    
    # Convert to matrix, transpose, then back to indices
    perm_matrix = perm_indices_to_perm_matrix(perm).T
    perm_to_apply = perm_matrix_to_perm_indices(perm_matrix)
    
    perms_to_apply[perm_name] = perm_to_apply

# Apply permutation to get n_permuted3 in universe space
universe_params = apply_permutation_to_statedict(
    perm_spec, perms_to_apply, n_permuted3.state_dict()
)

# Now transform from universe space to n3's space using n3's perm matrix (not transposed)
perms_to_apply = {}
for perm_name in perm_matrices[0]['a'].keys():
    # Get original permutation indices
    perm = perm_matrices[0]['a'][perm_name]
    
    # Convert to matrix (no transpose) then to indices
    perm_matrix = perm_indices_to_perm_matrix(perm)
    perm_to_apply = perm_matrix_to_perm_indices(perm_matrix)
    
    perms_to_apply[perm_name] = perm_to_apply

# Apply permutation to transform from universe to n3's space
transformed_params = apply_permutation_to_statedict(
    perm_spec, perms_to_apply, universe_params
)

# Load the transformed parameters
n_permuted3.load_state_dict(transformed_params)

# Check if results match n3
for layer in n3.state_dict():
    if not torch.allclose(n3.state_dict()[layer], n_permuted3.state_dict()[layer], rtol=1e-4, atol=1e-4):
        print(f'{layer} not the same')
        
        # Get the tensors
        tensor1 = n2.state_dict()[layer]
        tensor2 = n_permuted2.state_dict()[layer]
        
        # Check if this is a weight matrix with rows (2D or 4D tensor)
        if len(tensor1.shape) in [2, 4]:
            # For 4D convolutional weights, reshape to 2D for comparison
            if len(tensor1.shape) == 4:
                t1_reshaped = tensor1.reshape(tensor1.shape[0], -1)
                t2_reshaped = tensor2.reshape(tensor2.shape[0], -1)
            else:
                t1_reshaped = tensor1
                t2_reshaped = tensor2
                
            # Calculate row-wise cosine similarity
            for i in range(t1_reshaped.shape[0]):
                row_sim = torch.cosine_similarity(
                    t1_reshaped[i].flatten().unsqueeze(0), 
                    t2_reshaped[i].flatten().unsqueeze(0), 
                    dim=1
                )
                
                # Find if this row is similar to any row in the other tensor
                all_sims = []
                for j in range(t2_reshaped.shape[0]):
                    sim = torch.cosine_similarity(
                        t1_reshaped[i].flatten().unsqueeze(0),
                        t2_reshaped[j].flatten().unsqueeze(0),
                        dim=1
                    )
                    all_sims.append((j, sim.item()))
                
                # Sort by similarity and check if the best match is the same index
                all_sims.sort(key=lambda x: x[1], reverse=True)
                best_match_idx, best_match_sim = all_sims[0]
                
                # Only print details if the best match is not the same index
                # or if the similarity is below threshold
                if best_match_idx != i or row_sim < 0.99:
                    if row_sim < 0.99:  # Threshold for obviously different rows
                        print(f'  Row {i} has low similarity: {row_sim.item():.5f}')
                    
                    # Only print best matches if the best match isn't the same index
                    if best_match_idx != i:
                        print(f"  Row {i} best matches in other tensor:")
                        for j, sim in all_sims[:3]:
                            print(f"    Row {j}: similarity {sim:.5f}")
            
            # Now check column-wise similarity
            if len(tensor1.shape) == 2:  # Only for 2D matrices
                # Calculate column-wise cosine similarity
                for i in range(t1_reshaped.shape[1]):
                    col_sim = torch.cosine_similarity(
                        t1_reshaped[:, i].unsqueeze(0), 
                        t2_reshaped[:, i].unsqueeze(0), 
                        dim=1
                    )
                    
                    # Find if this column is similar to any column in the other tensor
                    all_sims = []
                    for j in range(t2_reshaped.shape[1]):
                        sim = torch.cosine_similarity(
                            t1_reshaped[:, i].unsqueeze(0),
                            t2_reshaped[:, j].unsqueeze(0),
                            dim=1
                        )
                        all_sims.append((j, sim.item()))
                    
                    # Sort by similarity and check if the best match is the same index
                    all_sims.sort(key=lambda x: x[1], reverse=True)
                    best_match_idx, best_match_sim = all_sims[0]
                    
                    # Only print details if the best match is not the same index
                    # or if the similarity is below threshold
                    if best_match_idx != i or col_sim < 0.99:
                        if col_sim < 0.99:  # Threshold for obviously different columns
                            print(f'  Column {i} has low similarity: {col_sim.item():.5f}')
                        
                        # Only print best matches if the best match isn't the same index
                        if best_match_idx != i:
                            print(f"  Column {i} best matches in other tensor:")
                            for j, sim in all_sims[:3]:
                                print(f"    Column {j}: similarity {sim:.5f}")
        
        print(f'n2[{layer}]={n2.state_dict()[layer]}')
        print(f'n_permuted2[{layer}]={n_permuted2.state_dict()[layer]}')
    else:
        print(f'{layer} is the same')
    similarity = torch.cosine_similarity(n3.state_dict()[layer].flatten(), n_permuted3.state_dict()[layer].flatten(), dim=0)
    print(f'{layer} similarity: {similarity:.5f})')


conv1.weight is the same
conv1.weight similarity: 1.00000)
conv1.bias is the same
conv1.bias similarity: 1.00000)
conv2.weight is the same
conv2.weight similarity: 1.00000)
conv2.bias is the same
conv2.bias similarity: 1.00000)
fc1.weight not the same
  Row 17 has low similarity: 0.08021
  Row 17 best matches in other tensor:
    Row 106: similarity 1.00000
    Row 40: similarity 0.17030
    Row 51: similarity 0.14671
  Row 30 has low similarity: 0.10414
  Row 30 best matches in other tensor:
    Row 17: similarity 1.00000
    Row 67: similarity 0.13691
    Row 77: similarity 0.12938
  Row 79 has low similarity: -0.08946
  Row 79 best matches in other tensor:
    Row 30: similarity 1.00000
    Row 4: similarity 0.12599
    Row 50: similarity 0.12106
  Row 106 has low similarity: -0.11349
  Row 106 best matches in other tensor:
    Row 79: similarity 1.00000
    Row 119: similarity 0.15562
    Row 43: similarity 0.12833
  Column 0 has low similarity: 0.94036
  Column 1 has low similarit